Tools that help with converting between molecular data formats (mol, xyz, SMILES)

from xyz to mols (from at, props)

In [ ]:
import os 

def xyz2mol(props):

    atomic_nums = props['_atomic_numbers']
    #make xyz block out of molecule for RDKit
    xyz_coords = props['_positions']

    xyz_string = f"{len(atomic_nums)}\n0.000 \n"
    for i in range(len(atomic_nums)):
        xyz_string += f"{atomic_nums[i]} {xyz_coords[i][0]:.6f} {xyz_coords[i][1]:.6f} {xyz_coords[i][2]:.6f}\n"


    #open temporary folder to write xyz into

    file = open('temp.xyz',mode='w')
    file.write(xyz_string)
    file.close()

    os.system('obabel temp.xyz -O temp.mol')
    
    
    mol_filename = 'temp.mol'
    return mol_filename

db file to SMILES strings data

In [ ]:
from schnetpack.datasets import QM9
from rdkit import Chem

db_filepath = '../data/datasets/QM9/qm9.db'
molfiles_dir = '../data/datasets/QM9/molfiles/'

range_molecules = [0,3500] 

data = QM9(db_filepath,download=False,remove_uncharacterized=True)

smiles_data = []

In [ ]:

molecules_pruned_smilesconversionfailed = [205,212,263,273,333,334,771,773,775,776,834,844,846,913,922,929,932,946,954,1045,1391,1451,1452,1596,1597,1598,1603,1623,3630,3631,3633,3691,3707,3744,3746]
for each_molecule in range(range_molecules[0],range_molecules[1]):
    if each_molecule not in molecules_pruned_smilesconversionfailed:
        print(each_molecule)
        at, props = data.get_properties(each_molecule)

        mol_filename = xyz2mol(props)

        mol = Chem.MolFromMolFile(mol_filename)
        smiles = Chem.MolToSmiles(mol)

        smiles_data.append([smiles,props['energy_U0'].detach().numpy()[0]])

print(smiles_data)

    

In [ ]:
import pandas as pd 

df = pd.DataFrame(smiles_data, columns=['SMILES', 'Energy'])
df.to_csv('../data/datasets/QM9/QM9SMILES/first3500-prunedfailedconver.csv',index=False)

Smiles to Mol

In [47]:
def xyz_element_to_atomicnumbers(xyz):

    elements_to_atomicnumb = {'H ': '1',
                              'C ': '6',
                              'N ': '7',
                              'O ': '8',
                              'F ': '9',
                              'P ': '15',
                              'S ': '16',
                              'Cl': '17',
                              'Br': '35',
                              'I ': '53',
                              '': '',
    }


    new_xyz = ''
    for line_index, each_line in enumerate(xyz.split('\n')):
        if line_index > 1:
            atomic_number = elements_to_atomicnumb[each_line[0:2]]
            each_line = each_line.replace(each_line[0:2],atomic_number+' ')

        new_xyz = new_xyz + each_line +'\n'

    return new_xyz
        

        



In [52]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd

smiles_filepath = '../Smiles2vec-master/data/IGC50.csv'

smiles_igc50 = pd.read_csv(smiles_filepath,delimiter=',')
smiles_list = smiles_igc50['smiles']

for each_smiles in range(2,3):

    # Read MOL file
    #mol_file_path = '../../dlchem/temp.mol'
    #mol = Chem.MolFromMolFile(mol_file_path)

    smiles = smiles_list[each_smiles]
    mol = Chem.MolFromSmiles(smiles)

    print(smiles)
    # Generate 3D coordinates
    mol = Chem.AddHs(mol)

    AllChem.EmbedMolecule(mol)
    AllChem.UFFOptimizeMolecule(mol)

    xyz = Chem.MolToXYZBlock(mol)
    
    xyz = xyz_element_to_atomicnumbers(xyz)
    print(xyz)
    output_mol_filename = "../Smiles2vec-master/data/igc50mols/%s.mol" %(each_smiles)
    output_xyz_filename = "../Smiles2vec-master/data/igc50mols/%s.xyz" %(each_smiles)
    # Write molecule to a .mol file
    writer = Chem.rdmolfiles.MolToMolFile(mol, output_mol_filename)  # Replace "output.mol" with your desired filename

    xyz_file = open(output_xyz_filename,mode='w')
    xyz_file.write(xyz)
    xyz_file.close()

OC(=O)C1=CC=CC(=C1Cl)Cl
15

8     -3.102560   -0.096676    0.094099
6     -2.050439    0.655546   -0.424991
8     -2.317822    1.667906   -1.124418
6     -0.640866    0.262966   -0.185215
6     -0.310553   -1.103628   -0.168674
6      1.007111   -1.514787    0.033580
6      2.013940   -0.570136    0.222506
6      1.710474    0.794054    0.214165
6      0.383625    1.222288    0.011617
17      0.020653    2.950479    0.069627
17      3.016942    1.945290    0.468858
1     -2.944115   -0.885115    0.709473
1     -1.068913   -1.858062   -0.336325
1      1.249213   -2.569801    0.037907
1      3.033309   -0.900325    0.377790
 



MOL to XYZ